<a href="https://colab.research.google.com/github/durfred/my-first-binder/blob/main/chess_RAG_NOGOOD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil

# 设置路径
CARLSEN_DATA_DIR = "/content/Carlsen"
CARLSEN_ZIP_FILENAME = "/content/Carlsen.zip"
PGN_FILE = f"{CARLSEN_DATA_DIR}/Carlsen.pgn"  # 解压后查看实际文件名

# --- 1. 清理旧文件 ---
print("\n--- 清理旧文件 ---")
shutil.rmtree(CARLSEN_DATA_DIR, ignore_errors=True)
if os.path.exists(CARLSEN_ZIP_FILENAME): os.remove(CARLSEN_ZIP_FILENAME)

# --- 2. 下载并解压 Carlsen 国际象棋数据 ---
print("\n--- 下载并解压棋谱数据 ---")
os.makedirs(CARLSEN_DATA_DIR, exist_ok=True)
!wget -O {CARLSEN_ZIP_FILENAME} https://www.pgnmentor.com/players/Carlsen.zip
!unzip -o {CARLSEN_ZIP_FILENAME} -d {CARLSEN_DATA_DIR}
os.remove(CARLSEN_ZIP_FILENAME)



--- 清理旧文件 ---

--- 下载并解压棋谱数据 ---
--2025-06-26 19:37:39--  https://www.pgnmentor.com/players/Carlsen.zip
Resolving www.pgnmentor.com (www.pgnmentor.com)... 65.254.227.240
Connecting to www.pgnmentor.com (www.pgnmentor.com)|65.254.227.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1553745 (1.5M) [application/zip]
Saving to: ‘/content/Carlsen.zip’

/content/Carlsen.zi 100%[===================>]   1.48M  2.94MB/s    in 0.5s    

2025-06-26 19:37:40 (2.94 MB/s) - ‘/content/Carlsen.zip’ saved [1553745/1553745]

Archive:  /content/Carlsen.zip
  inflating: /content/Carlsen/Carlsen.pgn  


In [2]:
!pip install python-chess llama-cpp-python --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 109.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4067781 sha256=c16521c0e2b5b75a6979212abfd15106a84571dfc1c574f54e8dca78fd0c3b61
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=b98679185fee58807bb4f35233ac0a048e940d6874ef23474676887ca997e774
  Stored in directory: /root/.cache/pip/wheels/fb/5d/5c/59a62d8a695285e59ec9c1f66add6f8a9ac4152499a2b

In [3]:
import chess.pgn

def extract_fens_and_moves_from_pgn(pgn_file, max_games=1000):
    fens = []
    moves = []

    with open(pgn_file) as f:
        game_count = 0
        while game_count < max_games:
            game = chess.pgn.read_game(f)
            if game is None:
                break

            board = game.board()
            for move in game.mainline_moves():
                fens.append(board.fen())
                moves.append(board.san(move))
                board.push(move)

            game_count += 1
    return fens, moves

fens, moves = extract_fens_and_moves_from_pgn(PGN_FILE)
print("总共提取棋盘状态:", len(fens))


总共提取棋盘状态: 86861


In [5]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 62.0 MB/s eta 0:00:00


In [6]:
import faiss
import numpy as np

def hash_fen(fen):
    return np.array([[hash(fen) % 1_000_000]], dtype='float32')  # shape=(1, 1)

index = faiss.IndexFlatL2(1)
for fen in fens:
    index.add(hash_fen(fen))

move_dict = dict(zip(range(len(moves)), moves))


In [7]:
def suggest_move_by_rag(current_fen):
    query = hash_fen(current_fen)
    D, I = index.search(query, 1)
    return move_dict[I[0][0]]



In [8]:
import chess.pgn

def extract_fens_and_moves_from_pgn(pgn_file, max_games=1000):
    fens = []
    moves = []

    with open(pgn_file) as f:
        game_count = 0
        while game_count < max_games:
            game = chess.pgn.read_game(f)
            if game is None:
                break

            board = game.board()
            for move in game.mainline_moves():
                fens.append(board.fen())
                moves.append(board.san(move))
                board.push(move)

            game_count += 1
    return fens, moves


In [9]:
!wget https://huggingface.co/codegood/gemma-2b-it-Q4_K_M-GGUF/resolve/main/gemma-2b-it.Q4_K_M.gguf

--2025-06-26 19:49:00--  https://huggingface.co/codegood/gemma-2b-it-Q4_K_M-GGUF/resolve/main/gemma-2b-it.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 3.168.73.38, 3.168.73.111, 3.168.73.129, ...
Connecting to huggingface.co (huggingface.co)|3.168.73.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/2e/3e/2e3e8837d8e4731552352bf441fc9458f6d4be48b3923aa2ea48e2107e8762bd/509635a3ad10c9d73a282e461ddb4415b68cbf2ce2f2c02cd0657cf933370d1a?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27gemma-2b-it.Q4_K_M.gguf%3B+filename%3D%22gemma-2b-it.Q4_K_M.gguf%22%3B&Expires=1750970940&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MDk3MDk0MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzJlLzNlLzJlM2U4ODM3ZDhlNDczMTU1MjM1MmJmNDQxZmM5NDU4ZjZkNGJlNDhiMzkyM2FhMmVhNDhlMjEwN2U4NzYyYmQvNTA5NjM1YTNhZDEwYzlkNzNhMjgyZTQ2MWRkYjQ0MTViNjhjYmYyY2UyZjJjMDJjZDA2NTdjZjk

In [10]:
from llama_cpp import Llama

llm = Llama(
    model_path="gemma-2b-it.Q4_K_M.gguf",
    n_ctx=2048,
    n_threads=8  # Increased threads
)

def get_move_from_llm(prompt):
    output = llm(prompt, max_tokens=128) # Increased max_tokens
    return output["choices"][0]["text"].strip()

llama_model_loader: loaded meta data with 24 key-value pairs and 164 tensors from gemma-2b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-2b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          gemma.block_count u32              = 18
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32              = 16384
llama_model_loader: - kv   6:                 gemma.attention.head_count u32              = 8
llama_model_loader: - kv   7:              gemma.attention.head_count_kv u32          

In [ ]:
import chess
import random # Import random for selecting a legal move if needed

def build_prompt_with_history(board, historical_move):
    fen = board.fen()

    # 构建人类风格棋谱历史（SAN 格式，如 1. e4 c5）
    move_history = []
    temp_board = chess.Board()
    for move in board.move_stack:
        move_history.append(temp_board.san(move))
        temp_board.push(move)

    numbered_history = ""
    for i in range(0, len(move_history), 2):
        turn = i // 2 + 1
        white = move_history[i]
        black = move_history[i+1] if i + 1 < len(move_history) else ""
        numbered_history += f"{turn}. {white} {black} "

    # Get legal moves in long algebraic notation
    legal_moves_uci = [move.uci() for move in board.legal_moves]
    legal_moves_str = ", ".join(legal_moves_uci)


    return f"""You are a chess AI assistant.

Game so far:
{numbered_history.strip()}

Current board position (FEN):
{fen}

A similar historical move was: {historical_move}

It is {'White' if board.turn else 'Black'} to play.

The legal moves in this position are: {legal_moves_str}

Please respond with ONLY the next legal move from the list provided above in **long algebraic notation** like "e2e4", "g1f3", or "c1d3".

Do NOT explain. Only output the move.

Move:"""

# 初始化棋盘
board = chess.Board()

# 主循环：玩家 vs AI
while not board.is_game_over():
    print("\n当前棋盘:")
    print(board)

    # 玩家走棋
    try:
        player_move = input("\n你的走法 (long algebraic 格式，如 e2e4): ").strip()
        move = chess.Move.from_uci(player_move)
        if move in board.legal_moves:
            board.push(move)
        else:
            print("❌ 非法走法:", player_move)
            continue
    except Exception as e:
        print("❌ 输入错误:", e)
        continue

    # Check if game is over after player's move
    if board.is_game_over():
        print("🏁 游戏结束！")
        break

    # AI 走棋
    fen = board.fen()
    query = hash_fen(fen)
    _, I = index.search(query, 1)
    historical_move = move_dict[I[0][0]]

    # Get AI suggestion
    prompt = build_prompt_with_history(board, historical_move)
    suggestion = get_move_from_llm(prompt).strip().split()[0]
    print("\n🤖 AI建议走法:", suggestion)

    # Validate AI move and push if legal, otherwise pick a random legal move
    try:
        ai_move = chess.Move.from_uci(suggestion)
        if ai_move in board.legal_moves:
            board.push(ai_move)
        else:
            print("❌ AI 建议非法走法:", suggestion, ". Choosing a random legal move.")
            legal_moves = list(board.legal_moves)
            if legal_moves:
                chosen_move = random.choice(legal_moves)
                print(f"🤖 AI makes legal move: {chosen_move.uci()}")
                board.push(chosen_move)
            else:
                print("🤷 No legal moves available for AI.")
                break # Should not happen in a standard game
    except Exception as e:
        print("❌ AI 输出格式错误:", suggestion, "| 错误:", e, ". Choosing a random legal move.")
        legal_moves = list(board.legal_moves)
        if legal_moves:
            chosen_move = random.choice(legal_moves)
            print(f"🤖 AI makes legal move: {chosen_move.uci()}")
            board.push(chosen_move)
        else:
            print("🤷 No legal moves available for AI.")
            break # Should not happen in a standard game


# 游戏结束
print("\n🏁 游戏结束！结果:", board.result())
print(board)


当前棋盘:
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R

你的走法 (long algebraic 格式，如 e2e4): d2d4


Llama.generate: 16 prefix-match hit, remaining 226 prompt tokens to eval
llama_perf_context_print:        load time =   20660.81 ms
llama_perf_context_print: prompt eval time =   73521.29 ms /   227 tokens (  323.88 ms per token,     3.09 tokens per second)
llama_perf_context_print:        eval time =     826.33 ms /     4 runs   (  206.58 ms per token,     4.84 tokens per second)
llama_perf_context_print:       total time =   20023.94 ms /   231 tokens



🤖 AI建议走法: e2e4
❌ AI 建议非法走法: e2e4 . Choosing a random legal move.
🤖 AI makes legal move: g7g5

当前棋盘:
r n b q k b n r
p p p p p p . p
. . . . . . . .
. . . . . . p .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B N R


KeyboardInterrupt: Interrupted by user

# Task
Improve the chess AI's ability to select legal and strategic moves by analyzing RAG effectiveness, improving prompt engineering, exploring alternative LLM strategies, implementing basic evaluation, and refining the AI move selection logic. Playtest and iterate on the improvements.

## Analyze rag effectiveness

### Subtask:
Evaluate if the current RAG approach (hashing FENs) is providing relevant historical moves. Consider alternative embedding methods for chess positions if needed.


**Reasoning**:
Evaluate the effectiveness of the current RAG approach by selecting a few FENs, hashing them, searching the index, retrieving the associated historical move, and manually inspecting the relevance of the suggested move.



In [ ]:
import chess

# Select a few diverse FENs from the list
selected_fens = [
    fens[0],       # Start of the game
    fens[100],     # Early game
    fens[1000],    # Mid game
    fens[10000],   # Later game
    fens[-1]       # End game (could be checkmate, draw, etc.)
]

print("--- Evaluating RAG Effectiveness ---")

for original_fen in selected_fens:
    print(f"\nOriginal FEN: {original_fen}")

    # Get hash and search index
    query = hash_fen(original_fen)
    D, I = index.search(query, 1)
    historical_index = I[0][0]

    # Retrieve historical move
    suggested_historical_move = move_dict[historical_index]

    # Note: Retrieving the historical FEN directly from the index using the index
    # is not straightforward with this simple hashing method and FAISS index.
    # We are only storing the index of the move.
    # For a more robust evaluation of the matched FEN, we would need to store
    # the FENs in the order they were added or use a different indexing method.
    # For now, we will focus on the suggested move's relevance.

    print(f"Index of closest historical FEN: {historical_index}")
    print(f"Suggested historical move: {suggested_historical_move}")

    # Manual Inspection Guidance:
    # Look at the Original FEN and the Suggested historical move.
    # Does this move make sense in the context of the original FEN board state?
    # With the current simple hashing, collisions are highly likely,
    # so the suggested move might be from a completely unrelated position.
    # This manual check helps confirm if the hashing method is effective
    # at finding truly *similar* historical positions.


--- Evaluating RAG Effectiveness ---

Original FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
Index of closest historical FEN: 0
Suggested historical move: d4

Original FEN: 2r2bk1/4nppp/4p3/p2pP3/n1r4P/PRP1B1P1/2N2P2/RN3K2 w - - 4 28
Index of closest historical FEN: 100
Suggested historical move: Bd4

Original FEN: 2r5/p1p1Rpk1/1p6/7b/2P3pp/BP1PK3/2P2P1P/8 b - - 3 45
Index of closest historical FEN: 1000
Suggested historical move: Kf6

Original FEN: r1bqr1k1/ppp2ppp/1nnp4/6b1/2PP4/1PN1BN1P/P4PP1/2RQKB1R w K - 1 12
Index of closest historical FEN: 10000
Suggested historical move: Nxg5

Original FEN: 2b3r1/pp6/2p2k2/2Pp1pN1/1P1RnP1r/P4R1P/6B1/6K1 b - - 0 38
Index of closest historical FEN: 86860
Suggested historical move: Rh5


## Analyze rag effectiveness

### Subtask:
Retry: Evaluate if the current RAG approach (hashing FENs) is providing relevant historical moves. Consider alternative embedding methods for chess positions if needed.


**Reasoning**:
The previous attempt showed that simple hashing of FENs is ineffective due to collisions. I need to find an alternative way to represent chess positions as numerical vectors (embeddings) that can be used with FAISS. I will implement a simple handcrafted feature-based embedding as a starting point. This involves counting pieces and their positions. I will then update the FAISS index and the RAG function to use this new embedding. Finally, I will perform a manual evaluation to check the relevance of the retrieved historical moves.



In [ ]:
import chess
import numpy as np
import faiss

def board_to_vector(board):
    """Converts a chess board state to a numerical vector (embedding)."""
    vector = np.zeros(768, dtype=np.float32) # 64 squares * 12 piece types = 768 features
    piece_map = {
        chess.PAWN: 0, chess.KNIGHT: 1, chess.BISHOP: 2,
        chess.ROOK: 3, chess.QUEEN: 4, chess.KING: 5
    }

    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            color_offset = 0 if piece.color == chess.WHITE else 6 # White pieces first 6, Black next 6
            piece_index = piece_map[piece.piece_type] + color_offset
            vector[square * 12 + piece_index] = 1 # One-hot encoding for piece on square

    # Add turn as a feature
    vector = np.append(vector, 1.0 if board.turn == chess.WHITE else 0.0) # 769 features

    # Add castling rights
    vector = np.append(vector, 1.0 if board.has_kingside_castling_rights(chess.WHITE) else 0.0) # 770
    vector = np.append(vector, 1.0 if board.has_queenside_castling_rights(chess.WHITE) else 0.0) # 771
    vector = np.append(vector, 1.0 if board.has_kingside_castling_rights(chess.BLACK) else 0.0) # 772
    vector = np.append(vector, 1.0 if board.has_queenside_castling_rights(chess.BLACK) else 0.0) # 773

    # Add en passant square
    en_passant_square = board.ep_square
    if en_passant_square is not None:
        vector = np.append(vector, en_passant_square + 1) # 774 (0 if no en passant)
    else:
        vector = np.append(vector, 0.0) # 774

    # Add halfmove clock (fifty-move rule)
    vector = np.append(vector, board.halfmove_clock) # 775

    # Add fullmove number
    vector = np.append(vector, board.fullmove_number) # 776 features in total


    return vector

# Generate embeddings for all FENs
fen_embeddings = np.array([board_to_vector(chess.Board(fen)) for fen in fens])

# Create a new FAISS index using the embeddings
embedding_dim = fen_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(fen_embeddings)

print(f"Created FAISS index with {index.ntotal} embeddings of dimension {embedding_dim}.")

def suggest_move_by_rag(current_fen):
    """Suggests a move based on the most similar historical position found via RAG."""
    current_board = chess.Board(current_fen)
    query_embedding = board_to_vector(current_board).reshape(1, -1) # Reshape for FAISS

    D, I = index.search(query_embedding, 1)
    closest_index = I[0][0]
    return move_dict[closest_index]

# Manual Evaluation (Repeat the evaluation from the previous step with new RAG)
print("\n--- Evaluating RAG Effectiveness with New Embeddings ---")

# Select a few diverse FENs from the list (using the same indices as before for comparison)
selected_indices = [0, 100, 1000, 10000, len(fens) - 1]
selected_fens = [fens[i] for i in selected_indices]

for original_fen in selected_fens:
    print(f"\nOriginal FEN: {original_fen}")

    # Get embedding and search index
    current_board = chess.Board(original_fen)
    query_embedding = board_to_vector(current_board).reshape(1, -1)
    D, I = index.search(query_embedding, 1)
    closest_index = I[0][0]

    # Retrieve historical move
    suggested_historical_move = move_dict[closest_index]

    # Retrieve the FEN of the closest historical position for better evaluation
    closest_fen = fens[closest_index]


    print(f"Index of closest historical FEN: {closest_index}")
    print(f"Closest historical FEN: {closest_fen}")
    print(f"Suggested historical move from historical FEN: {suggested_historical_move}")

    # Manual Inspection Guidance:
    # Compare the Original FEN with the Closest historical FEN.
    # Are they similar? Then look at the Suggested historical move.
    # Does this move make sense in the context of both the original and closest historical FEN?
    # This manual check helps confirm if the new embedding method is effective
    # at finding truly *similar* historical positions and if the suggested move is relevant.


Created FAISS index with 86861 embeddings of dimension 776.

--- Evaluating RAG Effectiveness with New Embeddings ---

Original FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
Index of closest historical FEN: 0
Closest historical FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
Suggested historical move from historical FEN: d4

Original FEN: 2r2bk1/4nppp/4p3/p2pP3/n1r4P/PRP1B1P1/2N2P2/RN3K2 w - - 4 28
Index of closest historical FEN: 100
Closest historical FEN: 2r2bk1/4nppp/4p3/p2pP3/n1r4P/PRP1B1P1/2N2P2/RN3K2 w - - 4 28
Suggested historical move from historical FEN: Bd4

Original FEN: 2r5/p1p1Rpk1/1p6/7b/2P3pp/BP1PK3/2P2P1P/8 b - - 3 45
Index of closest historical FEN: 1000
Closest historical FEN: 2r5/p1p1Rpk1/1p6/7b/2P3pp/BP1PK3/2P2P1P/8 b - - 3 45
Suggested historical move from historical FEN: Kf6

Original FEN: r1bqr1k1/ppp2ppp/1nnp4/6b1/2PP4/1PN1BN1P/P4PP1/2RQKB1R w K - 1 12
Index of closest historical FEN: 10000
Closest historical FEN: r1bqr1k1/ppp2ppp/

## Improve prompt engineering

### Subtask:
Experiment with different prompt structures and phrasings to better guide the LLM towards selecting legal and potentially more strategic moves from the provided list.


**Reasoning**:
I will analyze the current prompt structure and create a new function with an alternative prompt structure that places legal moves earlier and emphasizes selecting only from the list.



In [ ]:
import chess

def build_prompt_with_history_v1(board, historical_move):
    """Builds a prompt for the LLM with historical context."""
    fen = board.fen()

    # Build human-readable move history (SAN format, e.g., 1. e4 c5)
    move_history = []
    temp_board = chess.Board()
    for move in board.move_stack:
        move_history.append(temp_board.san(move))
        temp_board.push(move)

    numbered_history = ""
    for i in range(0, len(move_history), 2):
        turn = i // 2 + 1
        white = move_history[i]
        black = move_history[i+1] if i + 1 < len(move_history) else ""
        numbered_history += f"{turn}. {white} {black} "

    # Get legal moves in long algebraic notation
    legal_moves_uci = [move.uci() for move in board.legal_moves]
    legal_moves_str = ", ".join(legal_moves_uci)

    # Alternative prompt structure: Legal moves first, stronger emphasis
    return f"""You are a chess AI assistant.

Here are the legal moves in the current position: {legal_moves_str}

Please select ONLY one move from this list.

Game so far:
{numbered_history.strip()}

Current board position (FEN):
{fen}

A similar historical move was: {historical_move}

It is {'White' if board.turn else 'Black'} to play.

Your response MUST be ONLY the selected legal move in **long algebraic notation** (e.g., "e2e4", "g1f3", "c1d3"). Do NOT include any other text, explanation, or formatting.

Move:"""

# Keep the original prompt function for potential comparison (optional)
# def build_prompt_with_history_original(board, historical_move):
#     fen = board.fen()
#     move_history = []
#     temp_board = chess.Board()
#     for move in board.move_stack:
#         move_history.append(temp_board.san(move))
#         temp_board.push(move)
#     numbered_history = ""
#     for i in range(0, len(move_history), 2):
#         turn = i // 2 + 1
#         white = move_history[i]
#         black = move_history[i+1] if i + 1 < len(move_history) else ""
#         numbered_history += f"{turn}. {white} {black} "
#     legal_moves_uci = [move.uci() for move in board.legal_moves]
#     legal_moves_str = ", ".join(legal_moves_uci)
#     return f"""You are a chess AI assistant.

# Game so far:
# {numbered_history.strip()}

# Current board position (FEN):
# {fen}

# A similar historical move was: {historical_move}

# It is {'White' if board.turn else 'Black'} to play.

# The legal moves in this position are: {legal_moves_str}

# Please respond with ONLY the next legal move from the list provided above in **long algebraic notation** like "e2e4", "g1f3", or "c1d3".

# Do NOT explain. Only output the move.

# Move:"""


# Initialize board
board = chess.Board()

# Main loop: Player vs AI
print("\n--- Starting New Game with Updated Prompt ---")
while not board.is_game_over():
    print("\n当前棋盘:")
    print(board)

    # Player move
    try:
        player_move = input("\n你的走法 (long algebraic 格式，如 e2e4): ").strip()
        move = chess.Move.from_uci(player_move)
        if move in board.legal_moves:
            board.push(move)
        else:
            print("❌ 非法走法:", player_move)
            continue
    except Exception as e:
        print("❌ 输入错误:", e)
        continue

    # Check if game is over after player's move
    if board.is_game_over():
        print("🏁 游戏结束！")
        break

    # AI move
    fen = board.fen()
    # Use the updated suggest_move_by_rag with the new embedding
    historical_move = suggest_move_by_rag(fen)

    # Get AI suggestion using the new prompt structure
    prompt = build_prompt_with_history_v1(board, historical_move) # Use the new prompt function
    suggestion = get_move_from_llm(prompt).strip().split()[0]
    print("\n🤖 AI建议走法:", suggestion)

    # Validate AI move and push if legal, otherwise pick a random legal move
    try:
        ai_move = chess.Move.from_uci(suggestion)
        if ai_move in board.legal_moves:
            board.push(ai_move)
        else:
            print("❌ AI 建议非法走法:", suggestion, ". Choosing a random legal move.")
            legal_moves = list(board.legal_moves)
            if legal_moves:
                chosen_move = random.choice(legal_moves)
                print(f"🤖 AI makes legal move: {chosen_move.uci()}")
                board.push(chosen_move)
            else:
                print("🤷 No legal moves available for AI.")
                break # Should not happen in a standard game
    except Exception as e:
        print("❌ AI 输出格式错误:", suggestion, "| 错误:", e, ". Choosing a random legal move.")
        legal_moves = list(board.legal_moves)
        if legal_moves:
            chosen_move = random.choice(legal_moves)
            print(f"🤖 AI makes legal move: {chosen_move.uci()}")
            board.push(chosen_move)
        else:
            print("🤷 No legal moves available for AI.")
            break # Should not happen in a standard game


# Game over
print("\n🏁 游戏结束！结果:", board.result())
print(board)


--- Starting New Game with Updated Prompt ---

当前棋盘:
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R

你的走法 (long algebraic 格式，如 e2e4): e2e4


Llama.generate: 9 prefix-match hit, remaining 246 prompt tokens to eval
llama_perf_context_print:        load time =   20660.81 ms
llama_perf_context_print: prompt eval time =   20072.91 ms /   246 tokens (   81.60 ms per token,    12.26 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   20076.79 ms /   247 tokens


IndexError: list index out of range

**Reasoning**:
The AI output is causing an `IndexError` because `split()[0]` is being called on an empty string or a string with no spaces. This likely means the LLM is not returning the move in the expected format. I will modify the `get_move_from_llm` function to be more robust in parsing the LLM's output and handle cases where it doesn't return a move directly, potentially by looking for move patterns or defaulting to the historical move if the LLM output is unparseable.



In [ ]:
import chess
import random

def build_prompt_with_history_v1(board, historical_move):
    """Builds a prompt for the LLM with historical context."""
    fen = board.fen()

    # Build human-readable move history (SAN format, e.g., 1. e4 c5)
    move_history = []
    temp_board = chess.Board()
    for move in board.move_stack:
        move_history.append(temp_board.san(move))
        temp_board.push(move)

    numbered_history = ""
    for i in range(0, len(move_history), 2):
        turn = i // 2 + 1
        white = move_history[i]
        black = move_history[i+1] if i + 1 < len(move_history) else ""
        numbered_history += f"{turn}. {white} {black} "

    # Get legal moves in long algebraic notation
    legal_moves_uci = [move.uci() for move in board.legal_moves]
    legal_moves_str = ", ".join(legal_moves_uci)

    # Alternative prompt structure: Legal moves first, stronger emphasis
    return f"""You are a chess AI assistant.

Here are the legal moves in the current position: {legal_moves_str}

Please select ONLY one move from this list.

Game so far:
{numbered_history.strip()}

Current board position (FEN):
{fen}

A similar historical move was: {historical_move}

It is {'White' if board.turn else 'Black'} to play.

Your response MUST be ONLY the selected legal move in **long algebraic notation** (e.g., "e2e4", "g1f3", "c1d3"). Do NOT include any other text, explanation, or formatting.

Move:"""

def get_move_from_llm(prompt):
    """Gets a move suggestion from the LLM and attempts to parse it."""
    output = llm(prompt, max_tokens=128) # Increased max_tokens
    text_output = output["choices"][0]["text"].strip()

    # Attempt to parse the output to find a legal move
    legal_moves_uci = [move.uci() for move in board.legal_moves]

    # Simple split and check first word
    potential_move = text_output.split()[0] if text_output else ""
    if potential_move in legal_moves_uci:
        return potential_move

    # If the first word isn't a legal move, try to find any legal move in the output
    for move_uci in legal_moves_uci:
        if move_uci in text_output:
            return move_uci

    # If no legal move found in the output, return a placeholder or handle as needed
    # For now, let's return None and handle it in the main loop
    return None


# Keep the original prompt function for potential comparison (optional)
# def build_prompt_with_history_original(board, historical_move):
#     fen = board.fen()
#     move_history = []
#     temp_board = chess.Board()
#     for move in board.move_stack:
#         move_history.append(temp_board.san(move))
#         temp_board.push(move)
#     numbered_history = ""
#     for i in range(0, len(move_history), 2):
#         turn = i // 2 + 1
#         white = move_history[i]
#         black = move_history[i+1] if i + 1 < len(move_history) else ""
#         numbered_history += f"{turn}. {white} {black} "
#     legal_moves_uci = [move.uci() for move in board.legal_moves]
#     legal_moves_str = ", ".join(legal_moves_uci)
#     return f"""You are a chess AI assistant.

# Game so far:
# {numbered_history.strip()}

# Current board position (FEN):
# {fen}

# A similar historical move was: {historical_move}

# It is {'White' if board.turn else 'Black'} to play.

# The legal moves in this position are: {legal_moves_str}

# Please respond with ONLY the next legal move from the list provided above in **long algebraic notation** like "e2e4", "g1f3", or "c1d3".

# Do NOT explain. Only output the move.

# Move:"""


# Initialize board
board = chess.Board()

# Main loop: Player vs AI
print("\n--- Starting New Game with Updated Prompt ---")
while not board.is_game_over():
    print("\n当前棋盘:")
    print(board)

    # Player move
    try:
        player_move = input("\n你的走法 (long algebraic 格式，如 e2e4): ").strip()
        move = chess.Move.from_uci(player_move)
        if move in board.legal_moves:
            board.push(move)
        else:
            print("❌ 非法走法:", player_move)
            continue
    except Exception as e:
        print("❌ 输入错误:", e)
        continue

    # Check if game is over after player's move
    if board.is_game_over():
        print("🏁 游戏结束！")
        break

    # AI move
    fen = board.fen()
    # Use the updated suggest_move_by_rag with the new embedding
    historical_move = suggest_move_by_rag(fen)

    # Get AI suggestion using the new prompt structure
    prompt = build_prompt_with_history_v1(board, historical_move) # Use the new prompt function
    suggestion = get_move_from_llm(prompt) # Get suggestion from the modified function

    print("\n🤖 AI建议走法:", suggestion)

    # Validate AI move and push if legal, otherwise pick a random legal move
    try:
        if suggestion and chess.Move.from_uci(suggestion) in board.legal_moves:
            ai_move = chess.Move.from_uci(suggestion)
            board.push(ai_move)
        else:
            print("❌ AI 建议非法走法或输出无法解析:", suggestion, ". Choosing a random legal move.")
            legal_moves = list(board.legal_moves)
            if legal_moves:
                chosen_move = random.choice(legal_moves)
                print(f"🤖 AI makes legal move: {chosen_move.uci()}")
                board.push(chosen_move)
            else:
                print("🤷 No legal moves available for AI.")
                break # Should not happen in a standard game
    except Exception as e:
        print("❌ AI 输出格式错误或解析异常:", suggestion, "| 错误:", e, ". Choosing a random legal move.")
        legal_moves = list(board.legal_moves)
        if legal_moves:
            chosen_move = random.choice(legal_moves)
            print(f"🤖 AI makes legal move: {chosen_move.uci()}")
            board.push(chosen_move)
        else:
            print("🤷 No legal moves available for AI.")
            break # Should not happen in a standard game


# Game over
print("\n🏁 游戏结束！结果:", board.result())
print(board)


--- Starting New Game with Updated Prompt ---

当前棋盘:
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R

你的走法 (long algebraic 格式，如 e2e4): e2e4


Llama.generate: 254 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =   20660.81 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =     207.91 ms /     1 runs   (  207.91 ms per token,     4.81 tokens per second)
llama_perf_context_print:       total time =     210.85 ms /     2 tokens



🤖 AI建议走法: None
❌ AI 建议非法走法或输出无法解析: None . Choosing a random legal move.
🤖 AI makes legal move: a7a6

当前棋盘:
r n b q k b n r
. p p p p p p p
p . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R

你的走法 (long algebraic 格式，如 e2e4): c1d3
❌ 非法走法: c1d3

当前棋盘:
r n b q k b n r
. p p p p p p p
p . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R

你的走法 (long algebraic 格式，如 e2e4): c1d4
❌ 非法走法: c1d4

当前棋盘:
r n b q k b n r
. p p p p p p p
p . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R

你的走法 (long algebraic 格式，如 e2e4): c1d5
❌ 非法走法: c1d5

当前棋盘:
r n b q k b n r
. p p p p p p p
p . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R

你的走法 (long algebraic 格式，如 e2e4): c1d3
❌ 非法走法: c1d3

当前棋盘:
r n b q k b n r
. p p p p p p p
p . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R

你的走法 (long alge

Llama.generate: 34 prefix-match hit, remaining 227 prompt tokens to eval
llama_perf_context_print:        load time =   20660.81 ms
llama_perf_context_print: prompt eval time =   20294.55 ms /   227 tokens (   89.40 ms per token,    11.19 tokens per second)
llama_perf_context_print:        eval time =     408.05 ms /     2 runs   (  204.03 ms per token,     4.90 tokens per second)
llama_perf_context_print:       total time =   20712.24 ms /   229 tokens



🤖 AI建议走法: None
❌ AI 建议非法走法或输出无法解析: None . Choosing a random legal move.
🤖 AI makes legal move: g8h6

当前棋盘:
r n b q k b . r
. p p p p p p p
p . . . . . . n
. . . . . . . .
. . . . P . . .
. . N . . . . .
P P P P . P P P
R . B Q K B N R

你的走法 (long algebraic 格式，如 e2e4): d3e5
❌ 非法走法: d3e5

当前棋盘:
r n b q k b . r
. p p p p p p p
p . . . . . . n
. . . . . . . .
. . . . P . . .
. . N . . . . .
P P P P . P P P
R . B Q K B N R

你的走法 (long algebraic 格式，如 e2e4): c3d5


Llama.generate: 19 prefix-match hit, remaining 246 prompt tokens to eval
llama_perf_context_print:        load time =   20660.81 ms
llama_perf_context_print: prompt eval time =   20014.44 ms /   246 tokens (   81.36 ms per token,    12.29 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   20018.15 ms /   247 tokens



🤖 AI建议走法: None
❌ AI 建议非法走法或输出无法解析: None . Choosing a random legal move.
🤖 AI makes legal move: c7c5

当前棋盘:
r n b q k b . r
. p . p p p p p
p . . . . . . n
. . p N . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R . B Q K B N R


KeyboardInterrupt: Interrupted by user

In [18]:
import chess
import random
from llama_cpp import Llama

# 初始化模型
llm = Llama(
    model_path="./gemma-2b-it.Q4_K_M.gguf",
    n_ctx=1024,
    n_threads=4,
    n_gpu_layers=0,
    verbose=False,
)

def print_board(board):
    print("\n当前棋盘:")
    print(board.unicode(borders=True))
    print()

def is_valid_move(move_str, board):
    try:
        move = chess.Move.from_uci(move_str)
        return move in board.legal_moves
    except:
        return False

def format_chat_prompt(messages):
    prompt = "<bos>"
    for i, m in enumerate(messages):
        role = "user" if m["role"] == "user" else "model"
        prompt += f"<start_of_turn>{role}\n{m['content'].strip()}<end_of_turn>\n"
    prompt += "<start_of_turn>model\n"
    return prompt

def query_ai_move(board, history):
    messages = [{"role": "user", "content": history}]
    prompt = format_chat_prompt(messages)

    response = llm.create_completion(
        prompt=prompt,
        max_tokens=8,
        temperature=0.7,
        stop=["<end_of_turn>"],
    )
    text = response["choices"][0]["text"].strip().split()[0].lower()
    return text

def get_random_move(board):
    return random.choice(list(board.legal_moves)).uci()

# 主程序
board = chess.Board()
print("♟️ 新棋局开始，人类 vs Gemma AI")
print_board(board)

history = ""

while not board.is_game_over():
    user_move = input("你的走法 (如 e2e4): ").strip().lower()
    if not is_valid_move(user_move, board):
        print("⚠️ 非法走法，请重试")
        continue
    board.push_uci(user_move)
    history += f"{user_move} "

    print_board(board)

    ai_move = query_ai_move(board, history)

    if not is_valid_move(ai_move, board):
        print(f"⚠️ AI 输出非法内容: '{ai_move}'")
        ai_move = get_random_move(board)
        print("⚠️ AI 输出非法走法，使用随机合法走法")

    print(f"✅ AI 实际走法: {ai_move}")
    board.push_uci(ai_move)
    history += f"{ai_move} "
    print_board(board)

print("🏁 游戏结束")
print("最终结果:", board.result())


llama_context: n_ctx_per_seq (1024) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


♟️ 新棋局开始，人类 vs Gemma AI

当前棋盘:
  -----------------
8 |♜|♞|♝|♛|♚|♝|♞|♜|
  -----------------
7 |♟|♟|♟|♟|♟|♟|♟|♟|
  -----------------
6 |⭘|⭘|⭘|⭘|⭘|⭘|⭘|⭘|
  -----------------
5 |⭘|⭘|⭘|⭘|⭘|⭘|⭘|⭘|
  -----------------
4 |⭘|⭘|⭘|⭘|⭘|⭘|⭘|⭘|
  -----------------
3 |⭘|⭘|⭘|⭘|⭘|⭘|⭘|⭘|
  -----------------
2 |♙|♙|♙|♙|♙|♙|♙|♙|
  -----------------
1 |♖|♘|♗|♕|♔|♗|♘|♖|
  -----------------
   a b c d e f g h

你的走法 (如 e2e4): e2e4

当前棋盘:
  -----------------
8 |♜|♞|♝|♛|♚|♝|♞|♜|
  -----------------
7 |♟|♟|♟|♟|♟|♟|♟|♟|
  -----------------
6 |⭘|⭘|⭘|⭘|⭘|⭘|⭘|⭘|
  -----------------
5 |⭘|⭘|⭘|⭘|⭘|⭘|⭘|⭘|
  -----------------
4 |⭘|⭘|⭘|⭘|♙|⭘|⭘|⭘|
  -----------------
3 |⭘|⭘|⭘|⭘|⭘|⭘|⭘|⭘|
  -----------------
2 |♙|♙|♙|♙|⭘|♙|♙|♙|
  -----------------
1 |♖|♘|♗|♕|♔|♗|♘|♖|
  -----------------
   a b c d e f g h

⚠️ AI 输出非法内容: 'i'
⚠️ AI 输出非法走法，使用随机合法走法
✅ AI 实际走法: f7f5

当前棋盘:
  -----------------
8 |♜|♞|♝|♛|♚|♝|♞|♜|
  -----------------
7 |♟|♟|♟|♟|♟|⭘|♟|♟|
  -----------------
6 |⭘|⭘|⭘|⭘|⭘|⭘|⭘|⭘|
  -----------------
5 |⭘|⭘|⭘|⭘|⭘|♟|⭘|⭘|

KeyboardInterrupt: Interrupted by user